# Stage 8 Experiment: Log-Additive + Frequency Q

**Critical Research Question**: Does frequency-domain Q work with log-additive objectives?

## Problem Statement

We seek optimal low-rank factorization M ≈ PC where:
1. **Good reconstruction**: $\|M - PC\|_F^2$ is small
2. **Smooth SAXS profiles**: P profiles are well-behaved (usually not explicitly regularized)
3. **Smooth elution curves**: C profiles are smooth, measured by $\text{tr}(CQC^T)$

## Context

From previous stages:
- **Stage 5**: Additive + frequency Q → **90% success** (breakthrough!)
  - Objective: $f = \|M - PC\|^2 + \lambda \cdot \text{tr}(CQC^T)$ with frequency Q
- **Stage 7**: Log-additive + generic smoothness → **25% success** (only operator with success)
  - Objective: $f = \log(\|M - PC\|^2) + \lambda \cdot \log(\text{tr}(CQC^T))$ with generic Q

**Hypothesis**: Log-additive + frequency Q → **>50% success?**

If yes: We get the "best of both worlds":
- Adaptive gradients from log-additive
- Problem-informed regularization from frequency Q

## Objective Formulation

**Additive operator** (Stage 5):
$$f_{\text{add}} = \|M - PC\|_F^2 + \lambda \cdot \text{tr}(CQC^T)$$

**Log-additive operator** (Stage 7, this notebook):
$$f_{\text{log}} = \log(\|M - PC\|_F^2) + \lambda \cdot \log(\text{tr}(CQC^T))$$

Where:
- $\text{tr}(CQC^T)$ measures smoothness of C (concentration profiles)
- Q can be generic $(D^2)^T D^2$ or frequency-domain (Stage 5)
- Log-additive creates adaptive penalty: $\nabla_C \propto \frac{1}{\text{tr}(CQC^T)}$

**Key Question**: Do adaptive gradients interact constructively with frequency filtering?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import svd
from molass.SAXS.Models.Simple import guinier_porod

np.random.seed(42)
print("Stage 8: Testing log-additive + frequency Q combination")
print("="*70)

## Section 1: Generate Synthetic SEC-SAXS Data

**Realistic SEC-SAXS Setup**:

- **C profiles** (elution/concentration): Gaussian peaks from SEC column- This matches actual experimental SEC-SAXS data structure
- **P profiles** (SAXS): Guinier-Porod models with physical parameters

In [ ]:
# Synthetic SEC-SAXS dataset: M = PC where P, C are known
K = 80  # Number of frames (elution time dimension)
n_q = 50  # Number of q-points (scattering vector dimension)
n_components = 2

frames = np.arange(K)

# Ground truth C (concentration/elution profiles) - SEC physics
def gaussian_peak(x, center, width, height=1.0):
    return height * np.exp(-((x - center)**2) / (2 * width**2))

C_true = np.zeros((n_components, K))
C_true[0, :] = gaussian_peak(frames, center=35, width=4)  # Component 1: larger particle, elutes early, narrower
C_true[1, :] = gaussian_peak(frames, center=55, width=6)  # Component 2: smaller particle, elutes late, broader

# Normalize
C_true[0, :] = C_true[0, :] / C_true[0, :].sum()
C_true[1, :] = C_true[1, :] / C_true[1, :].sum()

# Ground truth P (SAXS profiles) - Guinier-Porod models
q = np.linspace(0.01, 0.3, n_q)  # Typical SAXS q-range (Å⁻¹)

# Component 1: LARGER particle → Rg = 40 Å, d = 4 (sphere)
G1 = 1.0  # Guinier prefactor
Rg1 = 40.0  # Radius of gyration (Å) - larger particle
d1 = 4.0  # Porod exponent (sphere)
p1_true = guinier_porod(q, G1, Rg1, d1)

# Component 2: SMALLER particle → Rg = 20 Å, d = 4 (sphere)
G2 = 1.0  # Same scale
Rg2 = 20.0  # Radius of gyration (Å) - smaller particle
d2 = 4.0  # Porod exponent (sphere)
p2_true = guinier_porod(q, G2, Rg2, d2)

P_true = np.vstack([p1_true, p2_true]).T  # (n_q, n_components)

# Generate observed data
M_clean = P_true @ C_true
noise_level = M_clean.mean() / 100  # SNR = 100
noise = np.random.randn(n_q, K) * noise_level
M = M_clean + noise
M = np.maximum(M, 0)  # Ensure non-negativity

print(f"Data shape: {M.shape} (q-points × frames)")
print(f"Components: {n_components}")
print(f"C_true peaks at frames: 35 (σ=4), 55 (σ=6)")
print(f"P_true: Component 1 Rg={Rg1}Å, Component 2 Rg={Rg2}Å")
print(f"SNR: {M_clean.mean() / noise_level:.0f}")

## Section 2: Create Frequency-Domain Q

Exact implementation from Stage 5 (where it achieved 90% with additive).

In [ ]:
def create_frequency_Q(K, peak_width, low_cutoff=0.1, high_cutoff=0.8):
    """
    Create Q-matrix based on frequency-domain filtering.
    
    Penalizes very low frequencies (constant/flat) and very high (noise).
    Allows mid-range frequencies corresponding to expected peak widths.
    
    Parameters
    ----------
    K : int
        Number of frames
    peak_width : float
        Expected peak width in frames
    low_cutoff : float
        Low frequency cutoff (0-1, fraction of max frequency)
    high_cutoff : float
        High frequency cutoff (0-1, fraction of max frequency)
    
    Returns
    -------
    Q : array (K, K)
        Frequency-based regularization matrix
    """
    # DCT basis
    F = np.zeros((K, K))
    for k in range(K):
        for n in range(K):
            F[k, n] = np.cos(np.pi * k * (n + 0.5) / K)
    F = F / np.sqrt(K / 2)
    F[0, :] /= np.sqrt(2)
    
    # Frequency weights (band-pass)
    freqs = np.arange(K) / K  # Normalized frequencies [0, 1]
    
    # Penalty for very low frequencies (constant/flat)
    low_penalty = np.exp(-((freqs - 0) / low_cutoff)**2)
    
    # Penalty for very high frequencies (noise/oscillations)
    high_penalty = 1 - np.exp(-((freqs - 1) / (1 - high_cutoff))**2)
    
    # Combined penalty (low in middle, high at extremes)
    weights = low_penalty + high_penalty
    weights[0] *= 10  # Extra penalty on DC component (flat)
    
    Lambda = np.diag(weights)
    
    # Q = F^T Λ F
    Q = F.T @ Lambda @ F
    
    return Q

# Create frequency Q with optimal parameters from Stage 5
Q_freq = create_frequency_Q(K, peak_width=5, low_cutoff=0.05, high_cutoff=0.7)

print(f"Frequency Q shape: {Q_freq.shape}")
print(f"Cutoffs: low=0.05, high=0.7 (optimal from Stage 5)")
print(f"\nThis Q achieved 90% success with additive objective.")
print(f"Will it work with log-additive?")

## Section 3: ALS with Log-Additive + Frequency Q

**Correct Objective** (consistent with Stage 7):
$$f = \log(\|M - PC\|_F^2) + \lambda \cdot \log(\text{tr}(CQC^T))$$

Where:
- First term: Log of reconstruction error (not raw squared error)
- Second term: Log of smoothness penalty on C
- This is the **log-additive operator** applied to (reconstruction, smoothness)

**Gradient structure**:
$$\frac{\partial f}{\partial C} = \frac{-2P^T(M - PC)}{\|M - PC\|_F^2} + \frac{\lambda}{\text{tr}(CQC^T)} \cdot 2CQ$$

Both terms have **adaptive inverse scaling** with their respective penalty magnitudes.

In [ ]:
def als_log_additive_frequency(M, k, Q, lam=0.1, max_iter=100, random_seed=42):
    """
    ALS with log-additive objective and frequency-domain Q.
    
    Objective: log(||M - PC||²) + λ·log(tr(CQC^T))
    
    This combines:
    - Adaptive gradients from log-additive (Stage 7)
    - Problem-informed regularization from frequency Q (Stage 5)
    
    Note: Simplified to focus on C regularization only (P not regularized)
    """
    np.random.seed(random_seed)
    n, m = M.shape
    
    # Initialize with SVD + small noise
    U, s, Vt = svd(M, full_matrices=False)
    P = np.abs(U[:, :k] @ np.diag(np.sqrt(s[:k]))) + 0.01
    C = np.abs(np.diag(np.sqrt(s[:k])) @ Vt[:k, :]) + 0.01
    
    history = {'recon': [], 'pen_C': [], 'total': []}
    
    for iteration in range(max_iter):
        # Update P - standard ALS without regularization
        # (could add log penalty on P, but focusing on C for degeneracy)
        CCT = C @ C.T
        P = M @ C.T @ np.linalg.inv(CCT + 1e-8 * np.eye(k))
        P = np.maximum(P, 1e-10)  # Ensure positivity
        
        # Update C with log-additive objective
        # Objective: log(||M-PC||²) + λ·log(tr(CQC^T))
        # Gradient wrt C:
        #   ∂f/∂C = -2/(||M-PC||²)·P^T(M-PC) + λ/(tr(CQC^T))·2CQ
        # 
        # For ALS update, we use gradient descent step or
        # approximate closed form by treating adaptive weights as constants
        
        PTP = P.T @ P
        PTM = P.T @ M
        
        # Current objective values for adaptive weights
        recon_error = np.linalg.norm(M - P @ C, 'fro')**2
        CQC = np.trace(C @ Q @ C.T)
        
        # Adaptive regularization weights (inverse scaling)
        alpha_recon = 1.0 / (recon_error + 1e-10)
        alpha_smooth = lam / (CQC + 1e-10)
        
        # Update each row of C independently
        # Balance: alpha_recon·(P^T P C = P^T M) + alpha_smooth·(C Q = 0)
        # Rearranged: (alpha_recon·P^T P + alpha_smooth·Q) C = alpha_recon·P^T M
        C_new = np.zeros_like(C)
        for i in range(k):
            # RHS: reconstruction contribution from other components
            rhs = alpha_recon * PTM[i, :]
            for j in range(k):
                if j != i:
                    rhs -= alpha_recon * PTP[i, j] * C[j, :]
            
            # LHS: (alpha_recon·PTP[i,i]·I + alpha_smooth·Q)
            lhs_matrix = alpha_recon * PTP[i, i] * np.eye(Q.shape[0]) + alpha_smooth * Q
            C_new[i, :] = np.linalg.solve(lhs_matrix + 1e-8 * np.eye(Q.shape[0]), rhs)
        
        C = np.maximum(C_new, 1e-10)  # Ensure positivity
        
        # Compute objective components (log-additive form)
        recon = np.linalg.norm(M - P @ C, 'fro')**2
        smoothness = np.trace(C @ Q @ C.T)
        
        log_recon = np.log(recon + 1e-10)
        log_smooth = np.log(smoothness + 1e-10)
        total = log_recon + lam * log_smooth
        
        history['recon'].append(recon)
        history['pen_C'].append(smoothness)
        history['total'].append(total)
        
        # Check convergence
        if iteration > 0:
            rel_change = abs(history['total'][-1] - history['total'][-2]) / (abs(history['total'][-2]) + 1e-10)
            if rel_change < 1e-6:
                break
    
    return P, C, history

print("ALS algorithm with log-additive + frequency Q defined")
print("Key features:")
print("  - Objective: log(||M-PC||²) + λ·log(tr(CQC^T))")
print("  - Both terms have adaptive inverse scaling")
print("  - Reconstruction weight: 1/||M-PC||²")
print("  - Smoothness weight: λ/tr(CQC^T)")
print("  - Frequency Q from Stage 5: band-pass filtering")

## Section 4: Evaluation Functions

Same evaluation as Stage 7 for direct comparison.

In [ ]:
def evaluate_solution(C_opt, C_true, corr_threshold=0.95):
    """
    Evaluate if recovered C matches ground truth.
    
    Returns
    -------
    is_correct : bool
        True if correlations > threshold (modulo permutation)
    is_degenerate : bool
        True if both components are nearly identical (all correlations > 0.95)
    """
    from scipy.stats import pearsonr
    
    k = C_opt.shape[0]
    
    # Check for degeneracy (all components identical)
    corr_matrix = np.zeros((k, k))
    for i in range(k):
        for j in range(k):
            corr_matrix[i, j] = abs(pearsonr(C_opt[i, :], C_opt[j, :])[0])
    
    # If all off-diagonal correlations are very high, it's degenerate
    off_diag = corr_matrix[np.triu_indices(k, k=1)]
    is_degenerate = np.all(off_diag > 0.95)
    
    if is_degenerate:
        return False, True
    
    # Check correlation with ground truth (try both permutations)
    corr_direct = [
        abs(pearsonr(C_opt[0, :], C_true[0, :])[0]),
        abs(pearsonr(C_opt[1, :], C_true[1, :])[0])
    ]
    
    corr_swapped = [
        abs(pearsonr(C_opt[0, :], C_true[1, :])[0]),
        abs(pearsonr(C_opt[1, :], C_true[0, :])[0])
    ]
    
    is_correct = (np.all(np.array(corr_direct) > corr_threshold) or
                  np.all(np.array(corr_swapped) > corr_threshold))
    
    return is_correct, False

print("Evaluation functions defined")
print("  - Correct: correlation > 0.95 (modulo permutation)")
print("  - Degenerate: both components nearly identical")

## Section 5: Multi-Start Experiment

Test log-additive + frequency Q with 20 random initializations.

**Comparison baselines**:
- Stage 5 (additive + frequency Q): 90%
- Stage 7 (log-additive + generic smoothness): 25%

**Hypothesis**: >50% success (best of both worlds)

In [ ]:
n_trials = 20
lambda_val = 0.1

print(f"Running multi-start experiment: {n_trials} trials")
print(f"Objective: log-additive + frequency Q")
print(f"λ = {lambda_val}")
print(f"Q = frequency (cutoffs: 0.05, 0.7 from Stage 5)")
print("="*70)

results = []

for trial in range(n_trials):
    P_opt, C_opt, history = als_log_additive_frequency(
        M, k=n_components, Q=Q_freq, lam=lambda_val, 
        max_iter=100, random_seed=trial
    )
    
    is_correct, is_degenerate = evaluate_solution(C_opt, C_true)
    
    results.append({
        'trial': trial,
        'correct': is_correct,
        'degenerate': is_degenerate,
        'final_objective': history['total'][-1],
        'n_iterations': len(history['total']),
        'C_opt': C_opt.copy()
    })
    
    status = "✓ CORRECT" if is_correct else ("✗ DEGENERATE" if is_degenerate else "✗ incorrect")
    print(f"Trial {trial:2d}: {status}  (obj={history['total'][-1]:8.2f}, iters={len(history['total']):3d})")

# Summary statistics
n_correct = sum(r['correct'] for r in results)
n_degenerate = sum(r['degenerate'] for r in results)
success_rate = n_correct / n_trials * 100

print("="*70)
print(f"\nRESULTS SUMMARY:")
print(f"  Correct: {n_correct}/{n_trials} ({success_rate:.0f}%)")
print(f"  Degenerate: {n_degenerate}/{n_trials} ({n_degenerate/n_trials*100:.0f}%)")
print(f"  Other failures: {n_trials - n_correct - n_degenerate}/{n_trials}")

print(f"\nCOMPARISON TO BASELINES:")
print(f"  Stage 5 (additive + frequency Q): 90%")
print(f"  Stage 7 (log-additive + generic):  25%")
print(f"  Stage 8 (log-additive + frequency): {success_rate:.0f}%")

if success_rate > 50:
    print(f"\n✓✓ HYPOTHESIS CONFIRMED! Best of both worlds achieved!")
    print(f"   Log-additive's adaptive gradients + frequency Q's problem-informed regularization = {success_rate:.0f}%")
elif success_rate > 25:
    print(f"\n⚠ PARTIAL SUCCESS: Better than generic smoothness ({success_rate:.0f}% vs 25%)")
    print(f"   But not as good as additive + frequency Q (90%)")
    print(f"   Possible explanation: Adaptive gradients reduce Q importance")
else:
    print(f"\n✗ NO IMPROVEMENT: Same as or worse than log-additive + generic ({success_rate:.0f}% vs 25%)")
    print(f"   Possible explanation: Frequency Q incompatible with log-additive?")

## Section 6: Visualize Representative Solutions

Compare correct vs incorrect solutions to understand what's happening.

In [ ]:
# Find one correct and one incorrect solution
correct_trials = [r for r in results if r['correct']]
incorrect_trials = [r for r in results if not r['correct'] and not r['degenerate']]
degenerate_trials = [r for r in results if r['degenerate']]

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Ground truth
ax = axes[0]
ax.plot(frames, C_true[0, :], 'b-', linewidth=2, label='Component 1')
ax.plot(frames, C_true[1, :], 'r-', linewidth=2, label='Component 2')
ax.set_title('Ground Truth', fontsize=12, fontweight='bold')
ax.set_xlabel('Frame')
ax.set_ylabel('Concentration')
ax.legend()
ax.grid(True, alpha=0.3)

# Correct solution (if any)
ax = axes[1]
if correct_trials:
    C_correct = correct_trials[0]['C_opt']
    ax.plot(frames, C_correct[0, :], 'b--', linewidth=2, alpha=0.7, label='Comp 1 (recovered)')
    ax.plot(frames, C_correct[1, :], 'r--', linewidth=2, alpha=0.7, label='Comp 2 (recovered)')
    ax.set_title(f'Correct Solution\n(Trial {correct_trials[0]["trial"]})', fontsize=12, fontweight='bold')
else:
    ax.text(0.5, 0.5, 'No correct solutions', ha='center', va='center', transform=ax.transAxes, fontsize=14)
    ax.set_title('Correct Solution (None found)', fontsize=12, fontweight='bold')
ax.set_xlabel('Frame')
ax.set_ylabel('Concentration')
ax.legend()
ax.grid(True, alpha=0.3)

# Incorrect or degenerate solution
ax = axes[2]
if degenerate_trials:
    C_bad = degenerate_trials[0]['C_opt']
    ax.plot(frames, C_bad[0, :], 'g-', linewidth=2, label='Comp 1')
    ax.plot(frames, C_bad[1, :], 'm-', linewidth=2, alpha=0.7, label='Comp 2')
    ax.set_title(f'Degenerate Solution\n(Trial {degenerate_trials[0]["trial"]})', fontsize=12, fontweight='bold')
elif incorrect_trials:
    C_bad = incorrect_trials[0]['C_opt']
    ax.plot(frames, C_bad[0, :], 'g-', linewidth=2, label='Comp 1')
    ax.plot(frames, C_bad[1, :], 'm-', linewidth=2, alpha=0.7, label='Comp 2')
    ax.set_title(f'Incorrect Solution\n(Trial {incorrect_trials[0]["trial"]})', fontsize=12, fontweight='bold')
else:
    ax.text(0.5, 0.5, 'All solutions correct!', ha='center', va='center', transform=ax.transAxes, fontsize=14, color='green')
    ax.set_title('Failure Mode (None found!)', fontsize=12, fontweight='bold')
ax.set_xlabel('Frame')
ax.set_ylabel('Concentration')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Visualization complete")

## Section 7: Frequency Analysis of Solutions

Examine frequency content of correct vs incorrect solutions.

**Key question**: Does frequency Q successfully enforce expected frequency structure in log-additive context?

In [ ]:
from scipy.fft import dct

def compute_frequency_spectrum(C):
    """Compute DCT frequency spectrum for each component."""
    spectra = []
    for i in range(C.shape[0]):
        spectrum = dct(C[i, :], norm='ortho')
        spectra.append(spectrum)
    return np.array(spectra)

if correct_trials and (incorrect_trials or degenerate_trials):
    # Get frequency spectra
    freq_true = compute_frequency_spectrum(C_true)
    freq_correct = compute_frequency_spectrum(correct_trials[0]['C_opt'])
    
    if degenerate_trials:
        freq_bad = compute_frequency_spectrum(degenerate_trials[0]['C_opt'])
        bad_label = 'Degenerate'
    else:
        freq_bad = compute_frequency_spectrum(incorrect_trials[0]['C_opt'])
        bad_label = 'Incorrect'
    
    # Plot frequency analysis
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    freqs = np.arange(K) / K
    
    for ax, spectrum, title in zip(axes, 
                                     [freq_true, freq_correct, freq_bad],
                                     ['Ground Truth', 'Correct Solution', f'{bad_label} Solution']):
        ax.plot(freqs, np.abs(spectrum[0, :]), 'b-', label='Component 1', linewidth=2)
        ax.plot(freqs, np.abs(spectrum[1, :]), 'r-', label='Component 2', linewidth=2, alpha=0.7)
        
        # Mark cutoff regions
        ax.axvspan(0, 0.05, alpha=0.2, color='red', label='Low cutoff (penalized)')
        ax.axvspan(0.7, 1.0, alpha=0.2, color='orange', label='High cutoff (penalized)')
        
        ax.set_xlabel('Normalized Frequency')
        ax.set_ylabel('|DCT Coefficient|')
        ax.set_title(title, fontsize=12, fontweight='bold')
        ax.set_yscale('log')
        ax.grid(True, alpha=0.3)
        if ax == axes[0]:
            ax.legend(fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    print("\nFREQUENCY ANALYSIS:")
    print("- Red shaded: DC component (flat, should be penalized)")
    print("- Orange shaded: High frequencies (noise, should be penalized)")
    print("- Middle range: Expected peak frequencies (should be allowed)")
    print("\nObservation: Do correct solutions have less energy in penalized regions?")
else:
    print("Cannot perform frequency analysis - need both correct and incorrect solutions")

## Section 8: Analysis and Interpretation

### Findings

**Result: 0% success (0/20 trials)** ❌

This is a **critical negative result**:
- **Stage 5** (additive + frequency Q): 90% ✓
- **Stage 7** (log-additive + generic Q): 25% ✓  
- **Stage 8** (log-additive + frequency Q): **0%** ✗

**Observations**:
1. All 20 trials failed to recover correct solution
2. All converged to identical objective value (1.46) at max iterations (100)
3. Recovered profiles show:
   - Peak locations approximately correct (frames 35, 55)
   - Peak shapes distorted with artifacts (extra bumps)
   - Components appear overly coupled/correlated
4. Algorithm appears stuck in poor local minimum
**This negative result is scientifically valuable** - it defines boundaries of the 3D design space (Operator × Q_form × Q_design).
### Implications for Stage 8 Framework
**Immediate priorities**:

1. **Understand the incompatibility mechanism**:
   - Does log(tr(CQC^T)) distort frequency structure?
   - Does adaptive scaling λ/tr(CQC^T) interfere with frequency filtering?
   - Visualize optimization trajectory to understand convergence failure

2. **Test fixed form theorem** (Stage 8 Question #1):
   - Current: $S(C) = \text{tr}(CQC^T)$ works for additive
   - Hypothesis: Log-additive may need $S_{\text{log}}(C) = \text{tr}(\log(C) \cdot Q \cdot \log(C)^T)$
   - Or: $S_{\text{log}}(C) = (\text{tr}(CQC^T))^{\alpha}$ with α ≠ 1

3. **Design operator-specific Q** (Stage 8 Question #3):
   - Frequency Q needs adaptation for log-additive context
   - May need to design Q in log-space rather than linear space
   - Explore: $Q_{\text{log}} = F^T \Lambda_{\text{log}} F$ where Λ_log accounts for log scaling

4. **Test other Q designs with log-additive**:
   - Spatial weighting (from Stage 5)
   - Combined spatial + ridge
   - Simple ridge regularization (baseline)
**If success >50%**:
**Boundary established**: Not all (Operator, Q_form, Q_design) combinations are compatible!


- Next: Understand why this combination works (gradient flow analysis)Then frequency Q design may need adaptation for log-additive context.

### Connection to Fixed Form Theorem (Stage 8 Question #1)
- Next: Test if other problem-informed Q designs (spatial weighting) also work with log-additive


$$S_{\text{log}}(C) = \log \text{tr}(CQC^T) \quad \text{or} \quad S_{\text{log}}(C) = \text{tr}(\log C \cdot Q \cdot \log C^T)$$

Current Q assumes $S(C) = \text{tr}(CQC^T)$ from Stage 4's additive objective.
**If success ~25-40%**:If log-additive requires different form:


- Partial improvement suggests some benefit from frequency Q

If log-additive requires different form:
- Next: Analyze why improvement is limited (adaptive gradients reduce Q importance?)Current Q assumes $S(C) = \text{tr}(CQC^T)$ from Stage 4's additive objective.

$$S_{\text{log}}(C) = \log \text{tr}(CQC^T) \quad \text{or} \quad S_{\text{log}}(C) = \text{tr}(\log C \cdot Q \cdot \log C^T)$$
- Next: Try tuning λ specifically for log-additive + frequency Q combination


### Connection to Fixed Form Theorem (Stage 8 Question #1)

Then frequency Q design may need adaptation for log-additive context.
**If success ≤25%**:

- No benefit from frequency Q in log-additive context- Next: Design Q specifically for log-additive (Stage 8 Question #3)
- Next: Investigate incompatibility (does log transform change frequency structure?)

## Summary

**Experiment**: Log-additive + frequency Q (Stage 8, Question #2)

**Baseline comparisons**:
- Additive + frequency Q (Stage 5): 90%
- Log-additive + generic Q (Stage 7): 25%

**Result**: **0% success (0/20 trials)** ❌

**Interpretation**: 
- **Incompatibility confirmed**: Frequency Q does NOT work with log-additive objectives
- Performs WORSE than log-additive + generic Q (0% vs 25%)

- Suggests destructive interference between:4. Map compatible vs incompatible regions of design space systematically

  - Adaptive gradients (∂f/∂B = λ/B) from log-additive3. Design log-specific Q: Adapt frequency filtering for log-space

  - Frequency-domain constraints from Q2. Test fixed form theorem: Does log-additive need different S(C) form?

- Log transform $\log(\text{tr}(CQC^T))$ may distort frequency properties1. Understand incompatibility mechanism (why 0% vs 25%?)

**Next steps**: 

**Critical insight**: The 3D design space (Operator × Q_form × Q_design) has **incompatible regions**. Not all combinations work!